In [4]:
# load input - output data and understand the structure

from scipy.io import loadmat

inputDataPath  = 'data/Proj2FeatVecsSet1.mat'
outputDataPath = 'data/Proj2TargetOutputsSet1.mat'

inputDataObj  = loadmat(inputDataPath)
outputDataObj = loadmat(outputDataPath)

inputData  = inputDataObj['Proj2FeatVecsSet1']
outputData = outputDataObj['Proj2TargetOutputsSet1']

data = zip(inputData, outputData)

In [5]:
"""

computes confusion matrix

@param   Y                   predicted labels

@param   ClassLabels         actual / true labels  

""" 

from sklearn.metrics import confusion_matrix

def MyConfusionMatrix(Y, ClassNames):
    return confusion_matrix(Y, ClassNames)    

In [13]:
"""

training script

"""

import numpy as np
import time

from sklearn.svm import SVC
from skrvm import RVC

from sklearn.model_selection import GridSearchCV

def MyTrainClassifier(XEstimate, XValidate, Parameters):
    
    X_train, Y_train = zip(*XEstimate)
        
    X_train = np.array(list(X_train))
    Y_train = np.array([np.where(output == 1)[0][0] for output in list(Y_train)])
    
    # sampling a small amount of training data for finding optimal hyper-parameters
    X_hyper = X_train[:2000, :]
    Y_hyper = Y_train[:2000]
    
    X_validate, Y_validate = zip(*XValidate)
        
    X_validate = list(X_validate)
    Y_validate = [np.where(output == 1)[0][0] for output in list(Y_validate)]
    
    # SVM
    # all vs all pair training
    if Parameters['algorithm'] == 'SVM':
        
        SVM(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate)
    
    elif Parameters['algorithm'] == 'RVM':
        
        RVM(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate)        
    
    elif Parameters['algorithm'] == 'Gaussian':
     
        Gaussian(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate)

In [14]:
def SVM(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate):
    
    hyper_param_grid = [
        {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
    ]

    estimator = GridSearchCV(SVC(decision_function_shape='ovo'), hyper_param_grid, cv=3, scoring='precision_macro')    
    estimator.fit(X_hyper, Y_hyper) 
    
    clf = estimator.best_estimator_
    print "found best estimator, training the estimator"
    
    clf.fit(X_train, Y_train)
    
    print "completed training"
    
    print clf.score(X_validate, Y_validate)

In [25]:
def RVM(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate):
    clf = RVC(n_iter=1, kernel='linear')

    start = time.clock()

    clf.fit(X_hyper, Y_hyper)
    print time.clock() - start, "s"

    print clf.score(X_validate, Y_validate)

In [26]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

def fact(n):
    if n == 0:
        return 1
    return n*fact(n-1)

def nCr(n, r):
    return fact(n)/(fact(n-r)*fact(r))

def extract2classes
def Gaussian(X_hyper, Y_hyper, X_train, Y_train, X_validate, Y_validate):
    print "gaussian in progress"
    
    kernel = 1.0 * RBF([1.0, 1.0])
    
    clf = GaussianProcessClassifier(kernel, multi_class='one_vs_one')
    clf.fit(X_hyper, Y_hyper)
    
    print clf.score(X_validate, Y_validate)

In [27]:
"""

 K-fold cross validation script

"""
from sklearn.model_selection import KFold
from random import shuffle

def MyCrossValidate(XTrain, Nf):
    shuffle(XTrain)
    kf = KFold(n_splits = Nf)
    
    for train_index, test_index in kf.split(XTrain):
        En = [XTrain[i] for i in train_index]
        Vn = [XTrain[i] for i in test_index]
        
        MyTrainClassifier(En, Vn, {'algorithm':'Gaussian'})

In [28]:
MyCrossValidate(data, 5)

gaussian in progress


ValueError: Anisotropic kernel must have the same number of dimensions as data (2!=60)

In [103]:
# RVM experiments

from skrvm import RVC
from sklearn.datasets import load_iris
import time

iris = load_iris()

clf = RVC(n_iter = 1)

start = time.clock()

clf.fit(iris.data, iris.target)
print time.clock() - start, "s"

clf.score(iris.data, iris.target)

0.382567 s


1.0

In [35]:

def Gaussian():
    

1
